In [3]:
from decisiontree import DecisionTree
from treeconstructor import C45Constructor, CARTConstructor
from ensemble import RFClassification
from collections import Counter
from sklearn.model_selection import StratifiedKFold, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [25]:
dbpedia_111 = pd.read_csv('datasets/dbpedia2014_dolce_hermit_11_prk.csv', sep='\t', index_col=[0]).fillna(0)
dbpedia_1111 = pd.read_csv('datasets/dbpedia2014_dolce_hermit_1111_rpk.csv', sep='\t', index_col=[0]).fillna(0)

feature_cols_111 = list(dbpedia_111.drop('Consistent', axis=1).columns)
feature_cols_1111 = list(dbpedia_1111.drop('Consistent', axis=1).columns)
label_col = 'Consistent'

print(Counter(dbpedia_111['Consistent'].values))

dbpedia_111_train = dbpedia_111.head(int(len(dbpedia_111)*0.8))
dbpedia_111_test = dbpedia_111.tail(int(len(dbpedia_111)*0.2))

c45 = C45Constructor()
cart = CARTConstructor()
rf = RFClassification()

Counter({True: 8, False: 3})


In [12]:
# Dataset Baseline Decision Tree Naive Bayes SVM Random Forest
# DBpedia-11 .727 .700 ± .458 600 ± .490 .600 ± .490 .600 ± .490
# DBpedia-111 .847 .937 ± .058 .756 ± .101 .946 ± .073 .901 ± .049
# DBpedia-1111 .877 .951 ± .016 .856 ± .043 .961 ± .015 .926 ± .017
# YAGO-11 .636 .550 ± .472 .850 ± .320 .650 ± .450 .650 ± .450
# YAGO-111 .739 .955 ± .061 .805 ± .153 .892 ± .066 .901 ± .064
# YAGO-1111 .714 .976 ± .015 .672 ± .056 .929 ± .052 .913 ± .034
# schema.org-11 .727 .700 ± .458 .600 ± 0.490 .700 ± .458 .600 ± .490
# schema.org-111 .645 .936 ± .071 .730 ± .158 .695 ± .066 .749 ± .182
# schema.org-1111 .649 .977 ± .020 .762 ± .066 .778 ± .060 .906 ± .038
# GoodRelations-11 .545 .700 ± .458 .700 ± .458 .700 ± .458 .700 ± .458
# GoodRelations-111 .604 .901 ± .076 .847 ± 0.129 .901 ± .076 .892 ± .078
# GoodRelations-1111 .631 .969 ± .016 .874 ± .027 .945 ± 0.20 .945 ± .027

In [13]:


tree = c45.construct_classifier(dbpedia_111_train, feature_cols_111, label_col)
tree.visualise('tree.pdf')
predictions = tree.evaluate_multiple(dbpedia_111_test.drop('Consistent', axis=1)).astype(int)
print(accuracy_score(dbpedia_111_test['Consistent'].astype(int), predictions))

tree = cart.construct_classifier(dbpedia_111_train, feature_cols_111, label_col)
tree.visualise('test.pdf')
predictions = tree.evaluate_multiple(dbpedia_111_test.drop('Consistent', axis=1)).astype(int)
print(accuracy_score(dbpedia_111_test['Consistent'].astype(int), predictions))

KernelException: 'orange.C45Learner': domain with discrete class attribute expected

In [30]:
trees = rf.construct_classifier(dbpedia_111_train, feature_cols_111, label_col)
predictions = trees.evaluate_multiple(dbpedia_111_test.drop('Consistent', axis=1)).astype(int)
print(accuracy_score(dbpedia_111_test['Consistent'].astype(int), predictions))

/usr/local/lib/python2.7/dist-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -1.15086268e-05]), 'nit': 5, 'funcalls': 49}
  " state: %s" % convergence_dict)
/usr/local/lib/python2.7/dist-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -1.08658701e-05]), 'nit': 5, 'funcalls': 49}
  " state: %s" % convergence_dict)
/usr/local/lib/python2.7/dist-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -3.82190124e-05]), 'nit': 5, 'funcalls': 49}
  " state: %s" % convergence_dict)


0.863636363636


In [15]:
dbpedia_111 = pd.read_csv('datasets/dbpedia2014_dolce_hermit_111_rpk.csv', sep='\t').fillna(0)
for cntr, statement in enumerate(dbpedia_111['Statement']):
    with open('aboxes/abox_dbpedia_'+str(cntr)+'.ttl', 'w') as fp:
        fp.write(statement)

In [19]:
skf = StratifiedKFold(n_splits=10)
loo = LeaveOneOut()
rf = RandomForestClassifier()

c45 = C45Constructor()
cart = CARTConstructor()
# rf = RFClassification()

dbpedia_111 = pd.read_csv('datasets/dbpedia2014_dolce_hermit_11_prk.csv', sep='\t', index_col=[0]).fillna(0)
my_dbpedia_111 = pd.read_csv('reconstructed_dbpedia_11.csv', index_col=[0, 1]).fillna(0)

my_accs = []
heiko_accs = []

for train_idx, test_idx in loo.split(dbpedia_111.drop('Consistent', axis=1)):
    heiko_train = dbpedia_111.iloc[train_idx, :]
    heiko_feature_cols = list(set(heiko_train.columns) - set(['Consistent']))
    heiko_test_X = dbpedia_111.drop('Consistent', axis=1).iloc[test_idx, :]
    heiko_test_y = dbpedia_111['Consistent'].iloc[test_idx]
    
    
    my_train = my_dbpedia_111.iloc[train_idx, :]
    my_feature_cols = list(set(my_train.columns) - set(['label']))
    my_test_X = my_dbpedia_111.drop('label', axis=1).iloc[test_idx, :]
    my_test_y = my_dbpedia_111['label'].iloc[test_idx]
    
    heiko_tree = cart.construct_classifier(heiko_train, heiko_feature_cols, 'Consistent')
#     heiko_tree.visualise('heiko_tree.pdf')
    
    my_tree = cart.construct_classifier(my_train, my_feature_cols, 'label')
#     my_tree.visualise('my_tree.pdf')
    heiko_acc = accuracy_score(heiko_test_y, heiko_tree.evaluate_multiple(heiko_test_X))
    heiko_accs.append(heiko_acc)
    my_acc = accuracy_score(my_test_y, my_tree.evaluate_multiple(my_test_X))
    my_accs.append(my_acc)
    print('Heiko accuracy:', heiko_acc)
    print('My accuracy:', my_acc)
print('-'*100)
print('Heiko mean acc:', np.mean(heiko_accs))
print('My mean acc:', np.mean(my_accs))

('Heiko accuracy:', 1.0)
('My accuracy:', 0.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 0.0)
('My accuracy:', 0.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 0.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 0.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 0.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 0.0)
----------------------------------------------------------------------------------------------------
('Heiko mean acc:', 0.63636363636363635)
('My mean acc:', 0.72727272727272729)


In [21]:
skf = StratifiedKFold(n_splits=10)
loo = LeaveOneOut()
rf = RandomForestClassifier()

c45 = C45Constructor()
cart = CARTConstructor()
# rf = RFClassification()

dbpedia_111 = pd.read_csv('datasets/dbpedia2014_dolce_hermit_111_rpk.csv', sep='\t', index_col=[0]).fillna(0)
my_dbpedia_111 = pd.read_csv('reconstructed_dbpedia_111.csv', index_col=[0]).fillna(0)

my_accs = []
heiko_accs = []

for train_idx, test_idx in loo.split(dbpedia_111.drop('Consistent', axis=1)):
    heiko_train = dbpedia_111.iloc[train_idx, :]
    heiko_feature_cols = list(set(heiko_train.columns) - set(['Consistent']))
    heiko_test_X = dbpedia_111.drop('Consistent', axis=1).iloc[test_idx, :]
    heiko_test_y = dbpedia_111['Consistent'].iloc[test_idx]
    
    
    my_train = my_dbpedia_111.iloc[train_idx, :]
    my_feature_cols = list(set(my_train.columns) - set(['label']))
    my_test_X = my_dbpedia_111.drop('label', axis=1).iloc[test_idx, :]
    my_test_y = my_dbpedia_111['label'].iloc[test_idx]
    
    heiko_tree = cart.construct_classifier(heiko_train, heiko_feature_cols, 'Consistent')
#     heiko_tree.visualise('heiko_tree.pdf')
    
    my_tree = cart.construct_classifier(my_train, my_feature_cols, 'label')
#     my_tree.visualise('my_tree.pdf')
    heiko_acc = accuracy_score(heiko_test_y, heiko_tree.evaluate_multiple(heiko_test_X))
    heiko_accs.append(heiko_acc)
    my_acc = accuracy_score(my_test_y, my_tree.evaluate_multiple(my_test_X))
    my_accs.append(my_acc)
    print('Heiko accuracy:', heiko_acc)
    print('My accuracy:', my_acc)
print('-'*100)
print('Heiko mean acc:', np.mean(heiko_accs))
print('My mean acc:', np.mean(my_accs))

('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 0.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 0.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 0.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accuracy:', 1.0)
('My accuracy:', 1.0)
('Heiko accur

In [22]:
heiko_tree.visualise('heiko_tree.pdf')
my_tree.visualise('my_tree.pdf')